## <CENTER>  [SWE] Anpassningar i ROOT: tips och exempel
    _________________________________________________
    
## <CENTER> [ENG] Fits in ROOT: tips and examples

In [ ]:
from ROOT import TH1F,TCanvas,TF1,TGraphErrors,TLegend,kRed,kBlue,kGreen,kBlack

### 2.1 [SWE] Osäkerheter
___________________________________
###  2.1 [ENG] Uncertainties
________________________

In [ ]:
# [SWE]: Skapa ett histogram 
# [ENG]: Create a histogram
histo = TH1F("histo", "; x; Number of events", 20, -3., 3.)
# [SWE]: antal entries
# [ENG]: number of entries
N = 500
# [SWE]: fyll histogrammet med slumpade tal från en funktion
# [ENG]: fill the histogram with randomised values from a function
fname="expo" # [SWE]: ta t.ex. "expo", en fördefinierad funktion med två parametrar = e^([parameter0] + [parameter1]*x). Andra exempel: "gaus", "landau"...
             # [ENG]: for example the pre-defined function "expo", with two parameters, = e^([parameter0] + [parameter1]*x). Other examples: "gaus", "landau"...
histo.FillRandom(fname, N) # [SWE]: fyll N gånger
                           # [ENG]: fill N times

In [ ]:
# [SWE]: aktivera interaktiv visualisering
# [ENG]: activate interactive visualisation
%jsroot on

In [ ]:
canvas = TCanvas("canvas", "", 800, 600)

In [ ]:
# [SWE]: Rita histogrammet
# [ENG]: Draw the histogram
histo.Draw("e1")
canvas.Draw()

#### [SWE] Ni ser att varje datapunkt har en osäkerhet. Ta reda på vad denna osäkerhet är lika med.

#### [ENG] Note that each point is drawn with an uncertainty. Find out what this uncertainty is equal to. 

In [ ]:
%run hints/hint2.py

___________________________________________
### 2.2 [SWE]  Gör en anpassning 
Nu ska vi göra en anpassningen till data i histogrammet ovan. Syntax är `TH1F::Fit(fcn, "fit_opt", "draw_opt", xlow, xup)`
där 
* `fcn` är antingen en sträng med namnet på en funktion (ROOT::TF1-objekt), eller själva funktionsobjektet.
* sedan kommer anpassnings- och plot-alternativ
* sist är range (nedre och övre gräns) i vilken anpassningen ska göras

Se vidare dokumentation här https://root.cern.ch/doc/master/classTH1.html#a7e7d34c91d5ebab4fc9bba3ca47dabdd. Framförallt kan man läsa om de olika alternativen för att kontrollera anpassningen (mer om detta snart).

Vi testar först att helt enkelt att anpassa till den fördefinierade funktionen "expo" som användes för att fylla histogrammet (borde per definition ge en bra anpassning).
Genom att ropa på Fit görs anpassningen och den resulterande kurvan ritas upp. 
Per default används en viktad minska-kvadrat-anpassning, även kallad $\chi^2$-anpassning (se "Statistik för fysikexperiment.pdf" på Canvas)
____________________________________________________________________________________

### 2.2  [ENG]  Make a fit
Now let's fit the data in the histogram. The syntax is `TH1F::Fit(fcn, "fit_opt", "draw_opt", xlow, xup)`
where
* `fcn` is either a string with the name of a function or a pointer to the function object itself. By function we mean a TF object (TF1 for a 1-dimensional fit)
* `"fit_opt"` are fit options, `"draw_opt"` are draw options. 
* `x_low` and `x_up` define the range to be used for the fit. 

See further documentation at https://root.cern.ch/doc/master/classTH1.html#a7e7d34c91d5ebab4fc9bba3ca47dabdd. You can read e.g. about the different fit options to control the fit (more on this soon)


We try to fit the same function we used to fill the histogram (should by definition work well).
By calling `histo.Fit` the fit is performed and the resulting curve is drawn in the canvas on top of the histogram.
By default a weighted least squares fit is used, also denoted $\chi^2$-fit.
___________________________________________

In [ ]:
histo.Fit(fname)
print("\n****** Fit results: ")
canvas.Draw() 

<b> [SWE]</b> Ovan ser ni resultaten från anpassningen. Titta t.ex. på `STATUS` som visar om anpassningen konvergerat (hittat minimum) eller inte. Ni ser de resulterade best-fit-värdena på parametrarna och deras respektive osäkerheter. Med fit option "V" skrivs mer detaljerad information ut.


Skriv ut den relativa osäkerheten på Slope-parametern nedan:
_____________________________________________________________
<b> [ENG]</b>: Above is the result from the fit. Look for example the `STATUS` which shows if the fit converged (found the minimum) or not. The best-fit values of the parameters and their corresponding uncertainties are printed. By passing fit option "V" more verbose output will be given.

Print out the relative uncertainty on the Slope parameter below:

In [ ]:
# print("Relative uncertainty Slope = ..."

#### [SWE]: Testa nu att gå tillbaka och öka antalet entries i histogrammet till det dubbla. Kör om anpassningen. Vad händer med osäkerheten? Om ni ökar med en faktor fyra? Förklara.
_____________________________________________________________
#### [ENG]: Now go back and increase the number of entries in the histogram to double and rerun the fit. What happens to the uncertainty? What if you increase by a factor of four? Explain.

___________________________________________________________________________

### 2.3   Goodness-of-fit
<b> [SWE] </b> Hur kan man kvantifiera hur bra en anpassning är?
Ett sätt är att utnyttja faktumet att minsta-kvadrat-summan under vissa villkor följer en $\chi^2$-fördelning.
Antalet frihetsgrader ($N_\rm{DOF}$) för fördelningen är lika med antalet datapunkter (binnar) minus antalet parametrar som ska anpassas.
Med detta kan man räkna ut sannolikheten att få det värde på summan som observerats.
Ett mer grovt mått är att jämföra minsta-kvadrat-summan med antalet frihetsgrader.
Kvoten mellan dem ska vara nära 1 för en bra anpassning.
Detta kan förstås genom att notera att varje datapunkt bör bidra med i snitt värdet 1 i summan.

Nedan tittar vi närmare på detta genom att göra en anpassning till en fyrkantspuls.
Se uppgiften längre ner.
___________________________________________________________________________
<b> [ENG] </b> How can we quantify how good a fit is? 
One way is to use the fact that the least squares sum follows a $\chi^2$ distribution, provided certain conditions are fulfilled.
The number of degress of freedom $N_\rm{DOF}$ is equal to the number of data points (bins) minus the number of parameters to be determined.
With this, the probability to get the observed value for the sum can be evaluated.
More roughly, one can consider the ratio between the sum and the number of degrees of freedom. 
Since each term in the sum should on average contribute around one, the ratio should be close to one for a good fit. 

Below we investigate this by making a fit to a square wave form.
A short assignment follows.



In [ ]:
import math
# [SWE] Gör en modell i form av en fyrkantsvåg
# [ENG] create a square wave form
squareWave = TH1F("squareWave", "; x; y", 3, 0.5, 3.5)
squareWave.SetBinContent(1, 5.); squareWave.SetBinError(1, math.sqrt(5.))
squareWave.SetBinContent(2, 15.); squareWave.SetBinError(2, math.sqrt(15.))
squareWave.SetBinContent(3, 5.); squareWave.SetBinError(3, math.sqrt(5.))
squareWave.SetLineColor(kBlack)

# [SWE] dra slumptal från fyrkantsvågen, dumpa värdena i ett nytt histogram
# [ENG] sample from it, dump it to a histogram
sample_squareWave = TH1F("sample_squareWave", "; x; y", 15, 0.5, 3.5)
n=2000
sample_squareWave.FillRandom(squareWave, n)

# [SWE] Rita upp vågen och de samplade värdena
# [ENG] Draw the original model and the sampled values
sample_squareWave.Draw("e1")
squareWave.Scale(float(n)/((15.+5.+5.)/3)/sample_squareWave.GetNbinsX()) 
squareWave.Draw("hist same")
canvas.Draw()

In [ ]:
# [SWE] Anpassa datapunkterna med en modell som är en konstant
# [ENG] Now let's make a fit using a constant
const = TF1("const", "[0]", 0.5, 3.5, 1)
const.SetParameter(0, 10.)
fitresult = sample_squareWave.Fit("const", "S", "SAME", 2.5, 3.5)
print("\n*** Chi-square sum = {:.1f}, Number of degrees of freedom = {}, ratio = {:.1f}".format(fitresult.Chi2(), fitresult.Ndf(), fitresult.Chi2()/fitresult.Ndf()))

canvas.Draw()

<b> [SWE] </b> Som väntat lägger anpassningen linjen någonstans mellan den nedre och den övre delen av pulsen.
Från  utskriften ser ni att $\chi^2/N_\rm{DOF}$ är långt ifrån 1–modellen klarar inte av att beskriva data.

<b> Testa nu att bara anpassa en del av pulsen där en konstant borde ge en bra anpassning.
    Räkna ut $\chi^2/N_\rm{DOF}$. Är den närmare 1? </b>
______________________________________________________

<b> [ENG] </b> As expected, the fit puts the line somewhere in between the upper and lower part of the pulse.
From the print-out we note that the $\chi^2/N_\rm{DOF}$ is far from one.
The model does not describe the data well.

<b> Now try to fit only a range in which a constant should be a good description of the data.
    Evaluate $\chi^2/N_\rm{DOF}$. Is it closer to one? </b>

____________________________________________________________________________
### 2.4 [SWE]  Konfigurering av anpassning
Default är som sagt att anpassa genom minimering av den viktade minsta-kvadrat-summan ($\chi^2$). Detta bygger på antagandet att mätvärdena kommer från normalfördelningen. I vårt fall gäller att de är Poissonfördelade–men vi vet att Poissonfördelningen närmar sig normalfördelningen då väntevärdet är stort.

Man kan också ge fit option = "L" för att göra en maximum likelihood-anpassning. I denna maximeras istället en likelihood som är lika med en produkt av Poisson-termer där varje term representerar antal counts i en bin.

Låt oss jämföra de två alternativen med ett exempel. Betrakta en modell som är en rät linje. Vi genererar ett antal event och ritar upp ett histogram, som motsvarar observationen av ett experiment. Målet är att mäta linjens lutning genom att göra en anpassning.
____________________________________________________________________________
### 2.4  [ENG]  Configuring the fit
By default, the fit is done by minimising the sum of weighted least squares ($\chi^2$). This is based on the assumption that the measured values come from a Gaussian distribution. In our case, they are counts, and come from the Poisson distribution. We know however that the Poisson distribution becomes increasingly Gaussian-like with increasing expectation value. 

You can give the fit option "L" to instead make a maximum likelihood fit. In this case, a likelihood function is maximised, where the function is equal to the product of Poisson terms. There is one term for each bin.  

Let's compare the two alternatives using an example. Consider a model equal to a straight line. We generate a number of events from the model and draw the histogram, corresponding to the observation made by an experiment. The goal is to measure the model's slope by performing a fit.
_______________________________________________________________________________

In [ ]:
# [SWE]: modell
# [ENG]: model
xlow=3.; xup=10.
fmodel = TF1("lin", "[0]*x+[1]", xlow, xup)
# [SWE]: fixera parametervärden
# [ENG]: set the parameter values
fmodel.SetParameters(1., 1.)
# [SWE]: skapa ett histogram
# [ENG]: create a histogram
hist = TH1F("exp", "; x; Number of events", 10, xlow, xup)
n=30
hist.FillRandom("lin", n) # SWE: fyll n gånger med slumpade värden från linjen
                          # ENG: fill histogram with n randomly generated numbers from the function

# [SWE]: anpassa med default minsta-kvadrat-method
# [ENG]: fit with the default least squares method
hist.Fit("lin", "", "e1")
# [SWE]: spara funktionen och rita upp den i blått (så att den inte försvinner när vi anpassar samma funktion igen senare)
# [ENG]: save the function and draw it again in blue (to prevent it from disappearing when drawing the same function again later)
fresultLeastSquares=hist.GetFunction("lin").Clone("linLeastSquares") # [SWE]: "Clone" skapar en kopia av funktionen
                                                                     # [ENG]: "Clone" creates a copy of the function
fresultLeastSquares.SetLineColor(kBlue)
fresultLeastSquares.Draw("SAME")
print("\nSlope least squares = {:.3f}".format(fresultLeastSquares.GetParameter(0)))

# [SWE]: anpassa igen men denna gång med option "L". Den resulterande kurvan kommer att visas i rött
# [ENG]: fit again, this time with option "L". The resulting curve is shown in red
hist.Fit("lin", "L", "e1 SAME")
print("Slope likelihood = {:.3f}\n".format(hist.GetFunction("lin").GetParameter(0)))
canvas.Draw()

_______________________________________________________________
#### [SWE]: Som ni ser får vi inte samma lutning med de olika alternativen för anpassning. Testa vad som händer om ni ökar antalet event i histogrammet. Förklara.

_________________
#### [ENG]: Note that the two curves to do not overlap. Try what happens if you increase the number of events. Explain.
_______________________________________________________________________________
_______________________________________________________________________________


### 2.5 [SWE] Definiera egna funktioner
____________________________________________________________________
### 2.5 [ENG] Define your own functions
______________________________________________________________________

In [ ]:
# [SWE] OBS interaktivitet funkar tyvärr inte för alla funktioner. Stäng av för att vara säker på att allt ritas som det ska.
# [ENG] NB the interactive feature unfortunately does not work for all functions. Turn off to make sure everything is drawn correctly.
%jsroot off

In [ ]:
# [SWE] definiera en egen funktion, utgå från a*(x-b)^2
# [ENG] define your own function, starting with a*(x-b)^2

### [SWE] Alternativ 1: skriv formeln direkt som en sträng med "TFormula"-syntax
### [ENG] Alternative 1: write the formula as a string using "TFormula"-syntax
f1 = TF1("f1", "[0]*(x-[1])*(x-[1])", -10., 10.) # [SWE]: [i] representerar parametrar, de två sista argumenten är range för funktionen
                                                 # [ENG]: [i] represent parameters, the last two arguments are the range for the function

### [SWE] Alternativ 2: skriv m.h.a. redan existerande funktioner (fördefinierade eller egendefinierade)
### [ENG] Alternative 2: write with already existing functions (pre-defined eller user defined)
f2 = TF1("f2", "gaus + f1", -10., 10.)

### [SWE]: Alternativ 3: med hjälp av egen-definierad python-funktion
### [ENG]: Alternative 3: using your own python function
def myfunc(x, params):
    x=x[0]
    a=params[0]; b=params[1]
    return a*(x-b)**2
f3 = TF1("f3", myfunc, -10., 10., 2) # [SWE] det sista argumentet anger antal parametrar
                                     # [ENG] the last argument specifies the number of parameters
### [SWE] sätt parameter-värden
### [ENG] set parameter values
f1.SetParameters(1., 2.)
f2.SetParameters(100., -2., 4., 1., 2.) # [SWE] första tre parametrarna är normalfördelningen, de sista f1-parametrarna
                                        # [ENG] the first three parameters are for the Gaussian, the last for the f1 function
f3.SetParameters(1., 3.)
### [SWE] sätt tvingande gränser för parametrarna (användbart vid anpassningar)
### [ENG] set limits for the parameters (that they are forced to stay inside) Useful to fitting
f1.SetParLimits(0, -3., 5.) # [SWE] parameter0 måste ligga inom [-3., 5.] 
                            # [ENG] parameter0 must lie inside [-3., 5.] 
### [SWE] function colors for drawing
### [ENG] colors for drawing
f1.SetLineColor(kBlue)
f2.SetLineColor(kBlack)
f3.SetLineColor(kRed)

### [SWE] Rita
### [ENG] Draw
f1.Draw()
f2.Draw("SAME")
f3.Draw("SAME")
canvas.Draw()

___________________________________________________

### 2.6 [SWE] Rita en legend
Generellt ska figurer ha en "legend" som visar vad de olika uppritade kurvorna/datapunkterna representerar. Nedan följer ett exempel. Se https://root.cern.ch/doc/master/classTLegend.html för ytterligare dokumentation.
_________________________________________________________________________________
### 2.6 [ENG] Draw a legend
In general, a figure should have a legend specifying what the different curves/histograms represent. Below is an example. See https://root.cern.ch/doc/master/classTLegend.html for further documentation.

In [ ]:
### [SWE] skapa legend-objektet
### [ENG] create the legend object
leg = TLegend(0.5, 0.5, 0.8, 0.8) # [SWE] argumenten är x1,y1,x2,y2-koordinater, givna som andel av canvas
                                  # [ENG] the arguments are x1,y1,x2,y2-coordinates, given as fraction of canvas
### [SWE] lägg till kurvorna vi ritade ovan. Syntax är TLegend::AddEntry(drawn_object, title, plot_style)
### [ENG] add the curves we drew above to the legend. Syntax is TLegend::AddEntry(drawn_object, title, plot_style)
leg.AddEntry(f1, "Parabola, defined with TFormula", "l") # "l" står för "line", alltså kurva
leg.AddEntry(f2, "Parabola + Gaussian", "l")
leg.AddEntry(f3, "Parabola, user defined", "l") # hade vi ritat ett histogram kan vi istället använda "p" 

## [SWE] gör legend snyggare
## [ENG] make legend prettier
# ...

## [SWE] rita legenden
## [ENG] draw legend
leg.Draw()
canvas.Draw()



_______________________________________________________________
[SWE] Se till att texten i legenden inte är för liten. Kan styras med `leg.SetTextSize(x)` där `x` är andel av canvas-storleken. Kanten kan tas bort med `leg.SetLegendBorderSize(0)`. Ändra i kodblocket ovan för att göra legenden snyggare.
_______________________________________________________________
[ENG] Make sure the font in the legend is not too small. This can be set with `leg.SetTextSize(x)` where `x` is fraction of canvas size. The legend border can be removed with `leg.SetLegendBorderSize(0)`. Change the code block above to make the legend prettier.
_______________________________________________________________

______________


### [SWE] Besvara följande frågor. Ni kommer att behöva dem för att i rapporten förklara hur anpassningen görs. 
* För histogram som ritats med något av `e`-alternativen har osäkerheter på varje datapunkt. ROOT har automatiskt räknat ut osäkerheterna till... vadå?
* Vad bestämmer osäkerheten på de anpassade parametrarna? Hur kan de minskas?
* ROOT anpassar per default med viktad minsta-kvadrat-metoden ($\chi^2$-anpassning). När är detta korrekt? Vad händer om vi anpassar med option "L"?

________________________________________________________________________________
### [ENG] Answer the following questions. You will need to understand them to properly explain the fit procedure when writing the report. 
* In a histogram, if drawn with with some `e` option, data points will be displayed with an uncertainty. ROOT has automatically evaluated the uncertainty to... what?
* What determines the uncertainty on the fitted parameters? How can they be reduced?
* ROOT uses the weighted least squares ($\chi^2$) fit by default. When is this appropriate? What happens if we use the option "L"?
